In [1]:
import pandas as pd
import os

In [43]:
iris = pd.read_csv("./data/IRIS.csv")

In [44]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [45]:
iris['species'].value_counts()

Iris-setosa        50
Iris-virginica     50
Iris-versicolor    50
Name: species, dtype: int64

In [46]:
# sepal과 petal들의 특징들을 이용하여 species를 예측해보자

# KNN 모델링
   + 1.학습데이터(train)와 예측데이터(test) 나누기
   + 2.학습데이터를 이용해 각 참조할 이웃의 수(k)별로 KNN모델 학습시키기
   + 3.만들어진 모델을 이용해 예측 데이터에서 예측
   + 4.예측 데이터에서 실제값과 예측치의 에러율을 살펴본다.
   + 5.에러율이 가장 작은 참조할 이웃의 수(k)를 이용한다.

## 고유 id가 없을 경우 고유 id 칼럼을 임의로 만들어주기

In [47]:
iris['id']=range(len(iris))

In [48]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species,id
0,5.1,3.5,1.4,0.2,Iris-setosa,0
1,4.9,3.0,1.4,0.2,Iris-setosa,1
2,4.7,3.2,1.3,0.2,Iris-setosa,2
3,4.6,3.1,1.5,0.2,Iris-setosa,3
4,5.0,3.6,1.4,0.2,Iris-setosa,4


In [49]:
iris=iris[['id','sepal_length','sepal_width','petal_length','petal_width','species']]
iris

,id,sepal_length,sepal_width,petal_length,petal_width,species
0,0,5.1,3.5,1.4,0.2,Iris-setosa
1,1,4.9,3.0,1.4,0.2,Iris-setosa
2,2,4.7,3.2,1.3,0.2,Iris-setosa
3,3,4.6,3.1,1.5,0.2,Iris-setosa
4,4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,145,6.7,3.0,5.2,2.3,Iris-virginica
146,146,6.3,2.5,5.0,1.9,Iris-virginica
147,147,6.5,3.0,5.2,2.0,Iris-virginica
148,148,6.2,3.4,5.4,2.3,Iris-virginica


## 학습데이터와 예측데이터(검증데이터) 나누기

### 학습데이터 만들기( train데이터 만들기)

In [50]:
#비복원추출로 해줘야 하므로 replace값은 False로 해줘야한다.
train = iris.sample(100,replace=False,random_state= 2020)
train.head()
# 무작위로 뽑다보니 id값이 순차적으로 나오지 않는다 이왕이면 순차적으로 나왔으면 좋겠다. (reset_index를 이용하면 된다.)


,id,sepal_length,sepal_width,petal_length,petal_width,species
104,104,6.5,3.0,5.8,2.2,Iris-virginica
8,8,4.4,2.9,1.4,0.2,Iris-setosa
61,61,5.9,3.0,4.2,1.5,Iris-versicolor
54,54,6.5,2.8,4.6,1.5,Iris-versicolor
78,78,6.0,2.9,4.5,1.5,Iris-versicolor


In [51]:
train = iris.sample(100,replace=False,random_state= 2020).reset_index()
train.head()
#순차적으로 인덱스가 되긴했는데 기존의 인덱스 값이 칼럼으로 생겨버렸으므로 drop시켜주자

,index,id,sepal_length,sepal_width,petal_length,petal_width,species
0,104,104,6.5,3.0,5.8,2.2,Iris-virginica
1,8,8,4.4,2.9,1.4,0.2,Iris-setosa
2,61,61,5.9,3.0,4.2,1.5,Iris-versicolor
3,54,54,6.5,2.8,4.6,1.5,Iris-versicolor
4,78,78,6.0,2.9,4.5,1.5,Iris-versicolor


In [52]:
train = iris.sample(100,replace=False,random_state= 2020).reset_index().drop(['index'],axis=1)
train

,id,sepal_length,sepal_width,petal_length,petal_width,species
0,104,6.5,3.0,5.8,2.2,Iris-virginica
1,8,4.4,2.9,1.4,0.2,Iris-setosa
2,61,5.9,3.0,4.2,1.5,Iris-versicolor
3,54,6.5,2.8,4.6,1.5,Iris-versicolor
4,78,6.0,2.9,4.5,1.5,Iris-versicolor
...,...,...,...,...,...,...
95,149,5.9,3.0,5.1,1.8,Iris-virginica
96,14,5.8,4.0,1.2,0.2,Iris-setosa
97,145,6.7,3.0,5.2,2.3,Iris-virginica
98,94,5.6,2.7,4.2,1.3,Iris-versicolor


### 예측 데이터(검증데이터) 만들기

In [53]:
test=iris.loc[~iris['id'].isin(train['id'])].reset_index().drop(['index'],axis=1)
test.head()

,id,sepal_length,sepal_width,petal_length,petal_width,species
0,1,4.9,3.0,1.4,0.2,Iris-setosa
1,3,4.6,3.1,1.5,0.2,Iris-setosa
2,6,4.6,3.4,1.4,0.3,Iris-setosa
3,9,4.9,3.1,1.5,0.1,Iris-setosa
4,20,5.4,3.4,1.7,0.2,Iris-setosa


## KNN모델링 하기  (분류형)

In [54]:
train.columns

Index(['id', 'sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')

In [55]:
from sklearn.neighbors import KNeighborsClassifier

In [56]:
knn=KNeighborsClassifier(n_neighbors=3)  # 참조할 이웃의 수는 일단 3개로 해서 한다

In [57]:
# .fit(input변수들 , target변수,)
knn.fit(train[['sepal_length','sepal_width','petal_length','petal_width']],train['species'])

KNeighborsClassifier(n_neighbors=3)

In [59]:
predictions =knn.predict(test[['sepal_length','sepal_width','petal_length','petal_width']])

In [60]:
test['pred'] = predictions

In [61]:
test.head()  # 정답인 species와 학습을 통한 예측인 pred가 많이 일치 할 수록 예측을 잘 한것이다.

,id,sepal_length,sepal_width,petal_length,petal_width,species,pred
0,1,4.9,3.0,1.4,0.2,Iris-setosa,Iris-setosa
1,3,4.6,3.1,1.5,0.2,Iris-setosa,Iris-setosa
2,6,4.6,3.4,1.4,0.3,Iris-setosa,Iris-setosa
3,9,4.9,3.1,1.5,0.1,Iris-setosa,Iris-setosa
4,20,5.4,3.4,1.7,0.2,Iris-setosa,Iris-setosa


In [63]:
(test['species']==test['pred']).mean()  #이렇게 하면 True인 값들의 비율을 구해준다

0.98

## 최적의 K값 찾아내기

In [64]:
#n_neighbors 값을 변화하게 되면 일치한 비율도 변화하게 된다.
knn=KNeighborsClassifier(n_neighbors=2)
knn.fit(train[['sepal_length','sepal_width','petal_length','petal_width']],train['species'])
predictions =knn.predict(test[['sepal_length','sepal_width','petal_length','petal_width']])
test['pred'] = predictions
(test['species']==test['pred']).mean()

0.94

In [65]:
knn=KNeighborsClassifier(n_neighbors=4)
knn.fit(train[['sepal_length','sepal_width','petal_length','petal_width']],train['species'])
predictions =knn.predict(test[['sepal_length','sepal_width','petal_length','petal_width']])
test['pred'] = predictions
(test['species']==test['pred']).mean()

0.98

### <strong>반복문으로 n_neighbors의 값을 바꿔가며 정답일치 비율을 살펴보고 최적의 K를 찾는다</strong>

In [69]:
for k in range(1,31):
    knn=KNeighborsClassifier(n_neighbors=k)
    knn.fit(train[['sepal_length','sepal_width','petal_length','petal_width']],train['species'])
    predictions =knn.predict(test[['sepal_length','sepal_width','petal_length','petal_width']])
    test['pred'] = predictions
    print((test['species']==test['pred']).mean())
# 가장 좋은 K는 그나마 가장 적은 3이 가장 좋은 것이다.

0.94
0.94
0.98
0.98
0.98
0.96
0.98
0.98
0.98
0.98
0.98
0.98
0.98
0.98
0.98
0.98
0.98
0.98
0.98
0.96
0.96
0.96
0.96
0.96
0.96
0.96
0.96
0.96
0.96
0.96


## Cross validation을 활용한 최적의 k 찾기

In [92]:
from sklearn.model_selection import cross_val_score
import numpy as np

In [95]:
knn=KNeighborsClassifier(n_neighbors=3)  
np.mean(cross_val_score(knn,iris[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']],iris['species'],cv=3))

0.9733333333333333

In [96]:
knn=KNeighborsClassifier(n_neighbors=2)  
np.mean(cross_val_score(knn,iris[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']],iris['species'],cv=3))

0.9466666666666667

In [98]:
for k in range(1,31):
    knn=KNeighborsClassifier(n_neighbors=k)
    print(np.mean(cross_val_score(knn,iris[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']],iris['species'],cv=3)))

# 어라? 최적의 K는 4가 되었네??? 

0.96
0.9466666666666667
0.9733333333333333
0.9866666666666667
0.98
0.9733333333333333
0.9733333333333333
0.9666666666666667
0.9733333333333333
0.9666666666666667
0.9666666666666667
0.9666666666666667
0.96
0.9533333333333333
0.9533333333333333
0.96
0.96
0.9533333333333333
0.9533333333333333
0.9466666666666667
0.9533333333333333
0.9466666666666667
0.94
0.9333333333333332
0.94
0.9533333333333333
0.9533333333333333
0.9533333333333333
0.9533333333333333
0.9533333333333333


## kNN 회귀모형

'sepal_length','sepal_width','petal_length' 이 3가지를 가지고 <strong>'petal_width'</strong>를 예측하는 회귀모형을 만들기  
+ 단, 회귀모형은 반드시 모두 수치형 데이터여야 한다 범주형은 제외해야한다.

In [71]:
train.head()
# 필요없는 범주형인 species는 지우기

,id,sepal_length,sepal_width,petal_length,petal_width,species
0,104,6.5,3.0,5.8,2.2,Iris-virginica
1,8,4.4,2.9,1.4,0.2,Iris-setosa
2,61,5.9,3.0,4.2,1.5,Iris-versicolor
3,54,6.5,2.8,4.6,1.5,Iris-versicolor
4,78,6.0,2.9,4.5,1.5,Iris-versicolor


In [72]:
del train['species']
del test['species']

In [73]:
train.head()

,id,sepal_length,sepal_width,petal_length,petal_width
0,104,6.5,3.0,5.8,2.2
1,8,4.4,2.9,1.4,0.2
2,61,5.9,3.0,4.2,1.5
3,54,6.5,2.8,4.6,1.5
4,78,6.0,2.9,4.5,1.5


In [75]:
from sklearn.neighbors import KNeighborsRegressor

In [78]:
knn= KNeighborsRegressor(n_neighbors=3)

In [79]:
knn.fit(train[['sepal_length','sepal_width','petal_length' ]],train['petal_width'])

KNeighborsRegressor(n_neighbors=3)

In [82]:
test['pred']=knn.predict(test[['sepal_length','sepal_width','petal_length']])

### mae
분류모델에서는 범주형 정답값과의 일치 여부(True여부)를 가지고 모델의 성능을 평가했다면  
회귀모형에서는 예측한 값과 실제 정답값과의 차이를 기준으로 성능을 평가한다. 즉, 차이값이 작을 수록 잘 예측했다고 할 수 있다  
이러한 지표를 우리는 mae라고 부른다.  

mae  
회귀문제에서 모델의 성능을 평가하는 방법 

In [84]:
test.head()

,id,sepal_length,sepal_width,petal_length,petal_width,pred
0,1,4.9,3.0,1.4,0.2,0.166667
1,3,4.6,3.1,1.5,0.2,0.200000
2,6,4.6,3.4,1.4,0.3,0.200000
3,9,4.9,3.1,1.5,0.1,0.133333
4,20,5.4,3.4,1.7,0.2,0.366667


In [88]:
abs(test['petal_width']-test['pred']).mean()  # 이와 같은 방법으로 mae를 구할 수 있다.

0.1673333333333333

In [91]:
for k in range(1,30):
    knn= KNeighborsRegressor(n_neighbors=k)
    knn.fit(train[['sepal_length','sepal_width','petal_length' ]],train['petal_width'])
    test['pred']=knn.predict(test[['sepal_length','sepal_width','petal_length']])
    print(abs(test['petal_width']-test['pred']).mean())
    
# k가 8일때 가장 잘 예측한다

0.196
0.16699999999999995
0.1673333333333333
0.1655
0.16839999999999994
0.1619999999999999
0.16342857142857137
0.16125
0.16155555555555556
0.16260000000000002
0.16254545454545455
0.1635
0.1636923076923077
0.1695714285714286
0.1738666666666667
0.17587499999999995
0.18282352941176455
0.18955555555555553
0.1913684210526316
0.19329999999999994
0.19780952380952382
0.20045454545454547
0.20330434782608697
0.2045
0.2084800000000001
0.2123076923076923
0.21599999999999997
0.21878571428571422
0.22365517241379315


최적의 K는 8이다!